In [ ]:
import tensorflow as tf
import preprocessing
import numpy as np
import reader

In [ ]:
class Config(object):
    """config."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 13
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000

In [ ]:
data_path = "/home/mor/projects/language_model/data"
save_path = "/home/mor/projects/language_model/save"

raw_data = reader.ptb_raw_data(data_path)
train_data, valid_data, test_data, _ = raw_data

config = Config()
eval_config = Config()
eval_config.batch_size = 1
eval_config.num_steps = 1

with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)

    with tf.name_scope("Train"):
        train_input = preprocessing.PTBInput(config=config, data=train_data, name="TrainInput")
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            m = preprocessing.PTBModel(is_training=True, config=config, input_=train_input)
        tf.contrib.deprecated.scalar_summary("Training Loss", m.cost)
        tf.contrib.deprecated.scalar_summary("Learning Rate", m.lr)

    with tf.name_scope("Valid"):
        valid_input = preprocessing.PTBInput(config=config, data=valid_data, name="ValidInput")
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mvalid = preprocessing.PTBModel(is_training=False, config=config, input_=valid_input)
        tf.contrib.deprecated.scalar_summary("Validation Loss", mvalid.cost)

    with tf.name_scope("Test"):
        test_input = preprocessing.PTBInput(config=eval_config, data=test_data, name="TestInput")
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mtest = preprocessing.PTBModel(is_training=False, config=eval_config,input_=test_input)

    sv = tf.train.Supervisor(logdir=save_path)
    with sv.managed_session() as session:
        for i in range(config.max_max_epoch):
            lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
            m.assign_lr(session, config.learning_rate * lr_decay)

            print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
            train_perplexity = preprocessing.run_epoch(session, m, eval_op=m.train_op,
                                                       verbose=True)
            print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
            valid_perplexity = preprocessing.run_epoch(session, mvalid)
            print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

        test_perplexity = preprocessing.run_epoch(session, mtest)
        print("Test Perplexity: %.3f" % test_perplexity)

        if save_path:
            print("Saving model to %s." % save_path)
            sv.saver.save(session, save_path, global_step=sv.global_step)

